In [1]:
#1. 사용환경 준비
import os
from dotenv import load_dotenv
from openai import OpenAI 

# openai API키 입력
load_dotenv()

client = OpenAI(api_key = os.getenv("OPENAI_API_KEY"))

In [2]:
#2. 모델 로드하기 
from langchain_openai import ChatOpenAI

# 모델 초기화
model = ChatOpenAI(model="gpt-4o-mini")

In [3]:
#3. 문서 로드하기
from langchain.document_loaders import CSVLoader

# PDF 파일 로드. 파일의 경로 입력
loader = CSVLoader("문화재_크롤링_결과.csv",encoding='UTF8')
loader2 = CSVLoader("여행지_크롤링_결과.csv",encoding='UTF8')

# 페이지 별 문서 로드
Treasures = loader.load()
Travel = loader2.load()

In [4]:
#4. 문서 청크로 나누기(CharacterTextSplitter)
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document



# 텍스트 청크 분할기 설정 (문단 기준 분할)
text_splitter = CharacterTextSplitter(
    
    separator="\n",
    chunk_overlap=0,
    length_function=len,
    is_separator_regex=False,
)

Treasures_splits = text_splitter.split_documents(Treasures) # 문서를 청크로 분할
Travel_splits = text_splitter.split_documents(Travel)

splits_sum = Treasures_splits + Travel_splits

In [5]:
#5 벡터 임베딩 생성
from langchain_openai import OpenAIEmbeddings

# OpenAI 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

In [6]:
#6. 벡터 스토어 생성
import faiss
from langchain_community.vectorstores import FAISS

# 문서에서 벡터 저장소 생성
vectorstore = FAISS.from_documents(documents=splits_sum, embedding=embeddings)


In [7]:
from langchain.vectorstores.base import VectorStore
#7. FAISS를 Retriever로 변환
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [8]:
#8. 프롬프트 템플릿을 정의
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# 프롬프트 템플릿 정의
contextual_prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the question using only the following context"
    """
    e.g. 국보 10호가 뭐야? : 국보 10호는 남원 실상사 백장암 삼층석탑입니다. 주소지는 전북 남원시 산내면 천왕봉로 447-76 입니다
    """
    """    
    보물 100호가 뭐야? : 보물 100호는 당진 안국사지 석조여래삼존입상. 주소지는 충남 당진시 정미면 원당골1길 188 (수당리) 입니다
    """
    """
    국보 10호 주변에 가볼만한 곳 추천해줘 : 국보 10호 주변에 가볼만한 곳은 광한루원,전북특별자치도 남원시 요천로 1447,063-620-8907 또는 지리산 허브밸리,전북특별자치도 남원시 운봉읍 바래봉길 214 또는 "인월시장 / 인월장(3, 8일)",전북특별자치도 남원시 인월면 인월로 65-3,063-625-1498 가 있습니다
    """
    ),
    ("user", "Context: {context}\\n\\nQuestion: {question}")
])

In [9]:
#9. LangChain의 모델과 프롬프트를 연결하여 RAG 체인을 구성
from langchain.chains import LLMChain

class SimplePassThrough:
    def invoke(self, inputs, **kwargs):
        return inputs

class ContextToPrompt:
    def __init__(self, prompt_template):
        self.prompt_template = prompt_template
    
    def invoke(self, inputs):
        # 문서 내용을 텍스트로 변환
        if isinstance(inputs, list):
            context_text = "\n".join([doc.page_content for doc in inputs])
        else:
            context_text = inputs
        
        # 프롬프트 템플릿에 적용
        formatted_prompt = self.prompt_template.format_messages(
            context=context_text,
            question=inputs.get("question", "")
        )
        return formatted_prompt

# Retriever를 invoke() 메서드로 래핑하는 클래스 정의
class RetrieverWrapper:
    def __init__(self, retriever):
        self.retriever = retriever

    def invoke(self, inputs):
        if isinstance(inputs, dict):
            query = inputs.get("question", "")
        else:
            query = inputs
        # 검색 수행
        response_Treasures = self.retriever.get_relevant_documents(query)
        return response_Treasures
#텍스트 생성 체인 생성
llm_chain = LLMChain(llm=model, prompt=contextual_prompt)

# RAG 체인 설정
rag_chain_debug = {
    "context": RetrieverWrapper(retriever),
    "prompt": ContextToPrompt(contextual_prompt),
    "llm": model
}

/var/folders/j9/nrfqzn3s2q349sv0cw_v64vm0000gn/T/ipykernel_23487/2174693903.py:40: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=model, prompt=contextual_prompt)


In [10]:
# 챗봇 구동
while True:
    print("========================")
    query = input("질문을 입력하세요 : ")
    if query == "종료": # 종료 입력 시 챗봇 종료
        break
    
    # 1. Retriever로 관련 문서 검색
    response_Treasures = rag_chain_debug["context"].invoke({"question": query})
    
    # 2. 문서를 프롬프트로 변환
    prompt_messages = rag_chain_debug["prompt"].invoke({
        "context": response_Treasures,
        "question": query
    })
    
    # 3. LLM으로 응답 생성
    response = rag_chain_debug["llm"].invoke(prompt_messages)
    
    print("\n질문:",query)
    print("\n답변:")
    print(response.content) # 답변 출력

/var/folders/j9/nrfqzn3s2q349sv0cw_v64vm0000gn/T/ipykernel_23487/2174693903.py:37: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response_Treasures = self.retriever.get_relevant_documents(query)



질문: 숭례문 주변 가볼만한곳 추천해줘

답변:
숭례문 주변에 가볼만한 곳은 국립중앙박물관, 서울특별시 용산구 서빙고로 137 또는 남산서울타워, 서울특별시 용산구 남산공원길 105가 있습니다.
